# 002 线性回归的简洁实现

In [22]:
# import package
import numpy as np
import torch
from torch.utils import data
from torch import nn


## 构造一个人造的数据集

### 定义线性模型参数

- Weight: $w=[2, -3.4]^T$
- Bias: $b=4.2$
- 噪声: $\epsilon$
- 标签: $y = Xw + b + \epsilon$

In [23]:
def create_data(w, b, size):
    # X: 均值为0, 方差为1, 大小为size*len(w)
    X = torch.normal(0, 1, (size, len(w)))
    y = torch.matmul(X, w) + b
    # 创造噪音epsilon
    epsilon = torch.normal(0, 0.01, y.shape)
    y += epsilon
    # reshape(x, y): 将张量变形，值为-1时表示自动.
    return X, y.reshape((-1, 1))


real_w, real_b = torch.tensor([2, -3.4]), 4.2
features, labels = create_data(w=real_w, b=real_b, size=1000)

features[0], labels[0]


(tensor([0.2110, 1.0073]), tensor([1.2043]))

### 调用API读取参数

In [24]:
def load_data(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


batch_size = 10
data_iter = load_data((features, labels), batch_size)

next(iter(data_iter))


[tensor([[-2.5490,  0.0678],
         [-0.6653,  0.0048],
         [-1.1237,  0.7159],
         [-1.2193,  0.4379],
         [ 1.4732,  0.5563],
         [ 1.2298,  0.0218],
         [ 0.5976,  0.8423],
         [ 1.4988, -0.5245],
         [ 0.2912, -0.3834],
         [ 0.7009, -0.7147]]),
 tensor([[-1.1301],
         [ 2.8543],
         [-0.4810],
         [ 0.2768],
         [ 5.2523],
         [ 6.5843],
         [ 2.5412],
         [ 8.9867],
         [ 6.0912],
         [ 8.0211]])]

## 定义模型

### 使用框架预定义好的层并初始化模型参数

In [25]:
net = nn.Sequential(nn.Linear(2, 1))
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)


tensor([0.])

### 定义Loss函数MSELoss

In [26]:
loss = nn.MSELoss()


### 实例化SGD实例

In [27]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)


## 开始训练

In [28]:
epoch_times = 10
for epoch in range(epoch_times):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch: {epoch+1}, loss: {l:f}')



epoch: 1, loss: 0.000170
epoch: 2, loss: 0.000100
epoch: 3, loss: 0.000101
epoch: 4, loss: 0.000100
epoch: 5, loss: 0.000101
epoch: 6, loss: 0.000101
epoch: 7, loss: 0.000102
epoch: 8, loss: 0.000100
epoch: 9, loss: 0.000100
epoch: 10, loss: 0.000100


In [32]:
w = net[0].weight.data
b = net[0].bias.data

diff_w = real_w - w.reshape(real_w.shape)
diff_b = real_b - b

diff_w, diff_b